In [ ]:
## Script to split the data into a federated Dataset
## Code from the same partitiondata.py file that was submitted in Part 1
## This will just create the dataset directory in the structure that we want and load the images
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import os



def split_data(dataset, num_clients, deviation_factor=0.1):
    """
    Splits data across clients with slight deviation
    """
    client_indices = {i: [] for i in range(num_clients)}

    # Split indices based on classes
    for class_idx in range(10):
        class_indices = np.where(np.array(dataset.targets) == class_idx)[0]
        np.random.shuffle(class_indices)

        # Calculate the number of samples
        total_samples = len(class_indices)
        avg_samples_per_client = total_samples // num_clients


        splits = []
        remaining_samples = total_samples
        for client in range(num_clients):
            # Apply deviation to the average samples
            deviation = np.random.randint(-int(avg_samples_per_client * deviation_factor),
                                          int(avg_samples_per_client * deviation_factor) + 1)
            client_sample_count = avg_samples_per_client + deviation

            if client == num_clients - 1:
                client_sample_count = remaining_samples
            else:
                client_sample_count = min(client_sample_count, remaining_samples)

            remaining_samples -= client_sample_count
            splits.append(client_sample_count)

        split_class_indices = np.split(class_indices, np.cumsum(splits[:-1]))

        for client, indices in enumerate(split_class_indices):
            client_indices[client].extend(indices)

    return client_indices



# Function to save client data as jpg
def save_client_data_as_jpg(dataset, indices, client_dir, client_id):
    client_path = os.path.join(client_dir, f'client_{client_id}')
    os.makedirs(client_path, exist_ok=True)


    for idx in indices:
        img, label = dataset[idx]
        label_folder = os.path.join(client_path, f'label_{label}')
        os.makedirs(label_folder, exist_ok=True)

        # Save the image as a .jpg
        img_path = os.path.join(label_folder, f'{idx}.jpg')

        img_pil = transforms.ToPILImage()(img)
        img_pil.save(img_path)





def main():
    transform = transforms.Compose([
                                transforms.RandomHorizontalFlip(p=0.3),
                                transforms.RandomApply([transforms.RandomRotation(10)], p=0.3),
                                transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
                                # transforms.RandomApply([transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))], p=0.2),
                                transforms.ToTensor(),
                                # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])

    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    num_clients = 10
    client_dir = './federated_clients_data/'

    # Spliting data for training and testing
    train_indices = split_data(train_dataset, num_clients, deviation_factor=0.2)
    test_indices = split_data(test_dataset, num_clients, deviation_factor=0.2)

    # Saving the  data for each client
    for client_id in range(num_clients):
        save_client_data_as_jpg(train_dataset, train_indices[client_id], client_dir, client_id)
        save_client_data_as_jpg(test_dataset, test_indices[client_id], client_dir, client_id)

    print(f"Data has been split and saved into '{client_dir}' for {num_clients} clients.")

if __name__ == '__main__':
    main()

100%|██████████| 170498071/170498071 [00:03<00:00, 55896301.90it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Data has been split and saved into './federated_clients_data/' for 10 clients.


In [ ]:
import torch

In [ ]:
print(torch.__version__)

2.4.1+cu121


In [ ]:
!pip install -q flwr[simulation] flwr-datasets[vision]

In [ ]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset
import matplotlib.pyplot as plt

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()

Training on cpu
Flower 1.12.0 / PyTorch 2.4.1+cu121


In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 32

In [ ]:
NUM_CLIENTS,BATCH_SIZE

(10, 32)

In [ ]:
import os
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image

In [ ]:
## Custom Dataset class that reads the data for the client from the appropriate directory based on the client ID provided
class ClientDataset(Dataset):
  def __init__(self,root_dir,client_id,transform=None):
    self.client_dir = os.path.join(root_dir,client_id)
    self.transform = transform
    self.images = []
    self.labels = []
    self._load_data()
  def _load_data(self):
    for label in os.listdir(self.client_dir):
            label_dir = os.path.join(self.client_dir, label)
            if os.path.isdir(label_dir):
                for img_file in os.listdir(label_dir):
                    img_path = os.path.join(label_dir, img_file)
                    self.images.append(img_path)
                    self.labels.append(int(label[-1]))

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_path = self.images[idx]
    image = Image.open(img_path)
    label = self.labels[idx]
    if self.transform:
        image = self.transform(image)
    return image, label


In [ ]:
## Function that is called by a Flower Framework Client to load the dataset
## Uses the Client Dataset class created above to using the specified client id derived from the partition id
def load_datasets(partition_id:int):
    root_dir = "/content/federated_clients_data"
    client_id = "client_" + str(partition_id)
    transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to match CIFAR-10 dimensions
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    client_dataset = ClientDataset(root_dir=root_dir, client_id=client_id, transform=transform)

    train_size = int(0.9 * len(client_dataset))
    test_size = len(client_dataset) - train_size

    train_dataset, test_dataset = random_split(client_dataset, [train_size, test_size])

    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset,[train_size,val_size])
    trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valloader = DataLoader(val_dataset,batch_size=BATCH_SIZE,shuffle=False)
    testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    return trainloader,valloader,testloader

In [ ]:
## Defines the neural network architecture that would be used by all the clients
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 32x32x3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # convolutional layer (sees 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 8x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, 10)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(-1, 64 * 4 * 4)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x

In [ ]:
import torch.optim as optim

In [ ]:
# Train and Test functions that are used by clients fit and evaluate methods
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            images, labels = batch
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item()*labels.size(0)
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch
            outputs = net(images)
            loss += criterion(outputs, labels).item()*labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total

    return loss, accuracy

In [ ]:
# Functions to get and set parameters for the model on clients
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [ ]:
# Global dictionaries to track the metrics across rounds for each client
accuracy_per_client_per_round = {}
loss_per_client_per_round = {}

In [ ]:
# Defines a flower client class
class FlowerClient(NumPyClient):
    def __init__(self, net, trainloader, valloader,partitionId):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.id = partitionId
        self.metrics_list = []

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        self.metrics_list.append((loss,accuracy))
        return float(loss), len(self.valloader), {"accuracy": float(accuracy),"loss": float(loss),"partition":str(self.id)}

In [ ]:
## A global cache to store the clients
clients_cache = {}

In [ ]:
# Function to create clients that will passed to the simulation so that the server can create clients as required
def client_fn(context: Context) -> Client:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)

    # Load data
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data partition
    partition_id = context.node_config["partition-id"]
    if partition_id not in clients_cache:
      trainloader, valloader, _ = load_datasets(partition_id=partition_id)
      clients_cache[partition_id] = FlowerClient(net, trainloader, valloader,partition_id).to_client()

    # FlowerClient is a subclass of NumPyClient, so we need to call .to_client()
    # to convert it to a subclass of `flwr.client.Client`
    return clients_cache[partition_id]


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [ ]:
# Custom Strategy Class to store metrics for each round into the global dictonaries
from flwr.server.strategy import FedAvg

class CustomStrategy(FedAvg):
  def aggregate_evaluate(self,rnd,results,failure):
    if failure:
      print(f"Round {rnd} had {len(failure)} failures")
    accuracy_list = [r.metrics["accuracy"] for _, r in results]

    for _,r in results:
      if r.metrics["partition"] not in accuracy_per_client_per_round:
        accuracy_per_client_per_round[r.metrics["partition"]] = []
      accuracy_per_client_per_round[r.metrics["partition"]].append((rnd,r.metrics["accuracy"]))
      if r.metrics["partition"] not in loss_per_client_per_round:
        loss_per_client_per_round[r.metrics["partition"]] =[]
      loss_per_client_per_round[r.metrics["partition"]].append((rnd,r.metrics["loss"]))

    average_accuracy = sum(accuracy_list) / len(accuracy_list)
    print(f"Round {rnd} accuracy: {average_accuracy}")
    print(f"Accuracy List for Round {rnd} : {accuracy_list}")


    return super().aggregate_evaluate(rnd, results, failure)

In [ ]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}

In [ ]:
# Global list for storing aggreagated metric
aggregate_metric_list = []

In [ ]:
# Function that calculates the average loss and accuracy
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    losses = [num_examples * m["loss"] for num_examples, m in metrics]  # For weighted loss
    examples = [num_examples for num_examples, _ in metrics]
    accuracy = sum(accuracies) / sum(examples)
    loss = sum(losses) / sum(examples)
    aggregate_metric_list.append({"accuracy": accuracy, "loss": loss})
    # Aggregate and return custom metric (weighted average)
    return {"accuracy": accuracy, "loss": loss}

In [ ]:
# Function that creates the Server object contains the Custom Stategy mentioned above
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.
    """

    strategy = CustomStrategy(
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=10,
        min_evaluate_clients=10,
        min_available_clients=10,
        evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
    )

    # Configure the server for 100 rounds of training
    config = ServerConfig(num_rounds=100)

    return ServerAppComponents(strategy=strategy, config=config)

In [ ]:
# Running the simulation
import os
os.environ["RAY_DEDUP_LOGS"] = "0"

# Create a new server instance with the updated FedAvg strategy
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

print(accuracy_per_client_per_round)
print(loss_per_client_per_round)
print(aggregate_metric_list)

DEBUG:flwr:Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=100, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=3836) 2024-10-15 04:09:07.777605: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=3836) 2024-10-15 04:09:07.825175: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=3836) 2024-10-15 04:09:07.837555: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=3835) 2024-10-15 04:09:11.557545: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

Round 1 accuracy: 0.1470869932305668
Accuracy List for Round 1 : [0.13508612873980055, 0.1382488479262673, 0.14574187884108866, 0.1503461918892186, 0.1476147614761476, 0.16074766355140188, 0.1484375, 0.15406162464985995, 0.12617702448210924, 0.16440831074977416]


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


Round 2 accuracy: 0.16984716414315731
Accuracy List for Round 2 : [0.17225747960108795, 0.17273576097105509, 0.18091809180918092, 0.1824884792626728, 0.1337099811676083, 0.17309594460929772, 0.15233644859813084, 0.17163504968383017, 0.193359375, 0.16593503072870938]


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


...

Round 99 accuracy: 0.600319630953447
Accuracy List for Round 99 : [0.5812115891132572, 0.6021602160216022, 0.6294930875576037, 0.5889792231255646, 0.620527306967985, 0.591588785046729, 0.6181998021760633, 0.5791015625, 0.5845004668534081, 0.6074342701722575]


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 100 round(s) in 7679.09s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.299668293169335
INFO :      		round 2: 2.2914812838720557
INFO :      		round 3: 2.2726917883506585
INFO :      		round 4: 2.2130174535219833
INFO :      		round 5: 2.1053064052619685
INFO :      		round 6: 2.0566827261168856
INFO :      		round 7: 2.0142492184509075
INFO :      		round 8: 1.967278967761017
INFO :      		round 9: 1.9586689611651815
INFO :      		round 10: 1.9297499043454538
INFO :      		round 11: 1.8974424077208218
INFO :      		round 12: 1.8782423070517023
INFO :      		round 13: 1.8447599915752566
INFO :      		round 14: 1.8192948615584819
INFO :      		round 15: 1.778203028294286
INFO :      		round 16: 1.7716956188556

Round 100 accuracy: 0.6129711349749842
Accuracy List for Round 100 : [0.615819209039548, 0.6065420560747663, 0.6145741878841089, 0.6310583580613254, 0.607421875, 0.6110599078341014, 0.6022408963585434, 0.6010879419764279, 0.6057605760576058, 0.6341463414634146]


INFO :      		round 89: 1.158032457892029
INFO :      		round 90: 1.1583098417593984
INFO :      		round 91: 1.1377240935995165
INFO :      		round 92: 1.1387669350918803
INFO :      		round 93: 1.1299932421736334
INFO :      		round 94: 1.1327513751048488
INFO :      		round 95: 1.1279608625754258
INFO :      		round 96: 1.106998534590681
INFO :      		round 97: 1.0967260367683984
INFO :      		round 98: 1.123763053468633
INFO :      		round 99: 1.1188964831211274
INFO :      		round 100: 1.1084146726702848
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, 0.147069217810153),
INFO :      	              (2, 0.16971204062171968),
INFO :      	              (3, 0.2016369944048468),
INFO :      	              (4, 0.21165053449559526),
INFO :      	              (5, 0.24524728427015202),
INFO :      	              (6, 0.256774130791081),
INFO :      	              (7, 0.2665318287723988),
INFO :      	              (8, 0.2816879605321602),
INFO :      	 

{'3': [(1, 0.13508612873980055), (2, 0.17225747960108795), (3, 0.19764279238440616), (4, 0.2157751586582049), (5, 0.26835902085222124), (6, 0.22937443336355395), (7, 0.26745240253853125), (8, 0.2629193109700816), (9, 0.2828649138712602), (10, 0.3019038984587489), (11, 0.30099728014505894), (12, 0.3082502266545784), (13, 0.3327289211242067), (14, 0.3272892112420671), (15, 0.3644605621033545), (16, 0.3535811423390752), (17, 0.37080689029918407), (18, 0.3653671804170444), (19, 0.37262012692656393), (20, 0.4043517679057117), (21, 0.40072529465095197), (22, 0.4125113327289211), (23, 0.4188576609247507), (24, 0.4442429737080689), (25, 0.44242973708068906), (26, 0.43064369900271987), (27, 0.4270172257479601), (28, 0.4496826835902085), (29, 0.44152311876699907), (30, 0.46600181323662737), (31, 0.4768812330009066), (32, 0.457842248413418), (33, 0.4524025385312783), (34, 0.46872166817769717), (35, 0.4551223934723481), (36, 0.4551223934723481), (37, 0.44786944696282865), (38, 0.4614687216681777),

In [ ]:
import pickle

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Saving the metric files to disk using pickle
with open('accuracy_per_client_per_round.pkl', 'wb') as f:
    pickle.dump(accuracy_per_client_per_round, f)
with open('loss_per_client_per_round.pkl','wb') as f:
    pickle.dump(loss_per_client_per_round,f)
with open('aggregate_metric_list.pkl','wb') as f:
    pickle.dump(aggregate_metric_list,f)